### Library Import

In [1]:
import os
import sys
from typing import List, Dict

import lightgbm as lgb
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

# Code 경로 추가
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(""))))
print(sys.path[-1])

c:\Users\xenx9\OneDrive\바탕화~1-DESKTOP-TT2HVSS-2792\Development\새 폴더\level1-classificationinmachinelearning-recsys-06


### Data Load

In [4]:
# 파일 호출
data_path: str = "../../data"
## raw.csv가 없는 경우 실행
# from Code.dataset.merge_all import merge_all
# df = merge_all(data_path)
df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "raw.csv"))
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv"))  # ID, target 열만 가진 데이터 미리 호출


### EDA (Explanatory Data Analysis)

In [5]:
eda_df = df.loc[df["_type"] == "train"]
# 각 열에서 누락된 값의 수를 계산
missing_values = eda_df.isnull().sum()

# 누락된 값의 백분율 계산
missing_percentage = (missing_values / len(eda_df)) * 100

# 누락된 값 비율을 기준으로 열 정렬
sorted_missing_percentage = missing_percentage.sort_values(ascending=False)
sorted_missing_percentage

hourly_market-data_liquidations_okex_btc_usd_long_liquidations_usd            100.0
hourly_market-data_open-interest_ftx_open_interest                            100.0
hourly_market-data_open-interest_ftx_btc_usd_open_interest                    100.0
hourly_market-data_taker-buy-sell-stats_okex_taker_sell_volume                100.0
hourly_market-data_taker-buy-sell-stats_okex_taker_buy_ratio                  100.0
                                                                              ...  
hourly_market-data_liquidations_htx_global_btc_usdt_short_liquidations_usd      0.0
hourly_market-data_liquidations_htx_global_btc_usdt_long_liquidations_usd       0.0
hourly_market-data_liquidations_htx_global_btc_usdt_short_liquidations          0.0
hourly_market-data_liquidations_htx_global_btc_usdt_long_liquidations           0.0
hourly_network-data_velocity_velocity_supply_total                              0.0
Length: 255, dtype: float64

In [5]:
# 몇 개 컬럼으로 EDA 예제
cols_dict: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_gap": "coinbase_premium_gap",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations": "long_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations": "short_liquidations",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_volume": "buy_volume",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_volume": "sell_volume",
}
# 추출후 단순화를 위해 rename
eda_df = eda_df[cols_dict.keys()].rename(cols_dict, axis=1)

In [6]:
# 타겟 레이블 분포
eda_df["target"].value_counts() / len(eda_df)

target
2.0    0.419064
1.0    0.404566
3.0    0.091895
0.0    0.084475
Name: count, dtype: float64

In [7]:
# 타겟 값 별로 변수의 평균 파익
bar_df = eda_df.groupby("target").agg({
    "coinbase_premium_gap": "mean",
    "long_liquidations": "mean",
    "short_liquidations": "mean",
    "buy_volume": "mean",
    "sell_volume": "mean",
}).reset_index()
bar_df

# 극단적인 경우, 즉 target 0, 3에서 평균적으로 coinbase_premium_gap, short_liquidations, buy_volume, sell_volume 값이 커짐

,target,coinbase_premium_gap,long_liquidations,short_liquidations,buy_volume,sell_volume
0,0.0,11.618797,34.306873,35.351660,4.046126e+08,4.037655e+08
1,1.0,5.919164,15.403795,17.878903,2.416815e+08,2.378602e+08
2,2.0,6.234038,20.308427,15.901697,2.380949e+08,2.442232e+08
3,3.0,13.476845,45.540121,37.819832,4.306805e+08,4.376228e+08


In [8]:
# 보다 관계를 명확히 파악하기 위해 barplot을 이용한 시각화 코드 작성
fig: go.Figure = make_subplots(
    rows=3,
    cols=2,
    shared_xaxes=True,
    subplot_titles=(
        "target distribution",
        "coinbase_premium_gap",
        "long_liquidations",
        "short_liquidations",
        "buy_volume",
        "sell_volume",
    ),
)
_target_disrtibution = eda_df["target"].value_counts() / len(eda_df)
fig.add_trace(
    go.Bar(
        x=_target_disrtibution.index,
        y=_target_disrtibution,
    ), row=1, col=1
)
fig.add_trace(go.Bar(x=bar_df["target"], y=bar_df["coinbase_premium_gap"]), row=1, col=2)
fig.add_trace(go.Bar(x=bar_df["target"], y=bar_df["long_liquidations"]), row=2, col=1)
fig.add_trace(go.Bar(x=bar_df["target"], y=bar_df["short_liquidations"]), row=2, col=2)
fig.add_trace(go.Bar(x=bar_df["target"], y=bar_df["buy_volume"]), row=3, col=1)
fig.add_trace(go.Bar(x=bar_df["target"], y=bar_df["sell_volume"]), row=3, col=2)
fig.update_layout(title_text="Target statistics", showlegend=False)
fig.update_xaxes(title_text="Target", row=3)
fig.show()

**위 그림으로부터 파악한 사실**

- target의 값 [0, 3]과 [1, 2]는 설명변수의 평균이 다르기 때문에 모델이 구분할 수 있을 것으로 추측
- 그러나, 각 target의 값 별로는 주어진 변수만으로 구별이 어려울 것으로 판단, 따라서 추가적인 feature engineering이 필요 

**가설**
1. long 포지션, short 포지션의 청산량의 차이가 가격 변동에 영향을 줄 것이다.
2. buy 거래량, sell 거래량의 차이가 가격 변동에 영향을 줄 것이다.

In [9]:
# liquidation, volume 으로 차이를 새로 생성하여 target 변수와의 관계 확인
eda_df = eda_df.assign(
    liquidation_diff=eda_df["long_liquidations"] - eda_df["short_liquidations"],
    volume_diff=eda_df["buy_volume"] - eda_df["sell_volume"],
)
bar_df = eda_df.groupby("target").agg({
    "liquidation_diff": "mean",
    "volume_diff": "mean",
}).reset_index()
bar_df  # target 별로 liquidation diff, volume diff 가 각각 평균적으로 양의 관계, 음의 관계를 갖고 있음을 확인

,target,liquidation_diff,volume_diff
0,0.0,-1.044786,8.470791e+05
1,1.0,-2.475109,3.821241e+06
2,2.0,4.406731,-6.128251e+06
3,3.0,7.720289,-6.942215e+06


In [10]:
# 보다 관계를 명확히 파악하기 위해 barplot을 이용한 시각화 코드 작성
fig: go.Figure = make_subplots(
    rows=1,
    cols=2,
    shared_xaxes=True,
    subplot_titles=(
        "liquidation_diff",
        "volume_diff",
    ),
)
fig.add_trace(go.Bar(x=bar_df["target"], y=bar_df["liquidation_diff"]), row=1, col=1)
fig.add_trace(go.Bar(x=bar_df["target"], y=bar_df["volume_diff"]), row=1, col=2)
fig.update_xaxes(title_text="Target")
fig.update_layout(title_text="Target statistics", showlegend=False)
fig.show()


**위 그림으로부터 파악한 사실**
- 가격 변동량이 클 때, 청산량의 차이 (long 청산 - short 청산)가 평균적으로 크다.
- 가격 변동량이 클 때, 거래량의 차이 (buy 거래량 - sell 거래량)가 평균적으로 작아진다.

따라서, 해당변수의 도입이 모델에 유의미한 영향을 줄 것이라 판단


### Feature engineering

In [11]:
# 모델에 사용할 컬럼, 컬럼의 rename rule을 미리 할당함
cols_dict: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_gap": "coinbase_premium_gap",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_index": "coinbase_premium_index",
    "hourly_market-data_funding-rates_all_exchange_funding_rates": "funding_rates",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations": "long_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations_usd": "long_liquidations_usd",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations": "short_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations_usd": "short_liquidations_usd",
    "hourly_market-data_open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_ratio": "buy_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_sell_ratio": "buy_sell_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_volume": "buy_volume",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_ratio": "sell_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_volume": "sell_volume",
    "hourly_network-data_addresses-count_addresses_count_active": "active_count",
    "hourly_network-data_addresses-count_addresses_count_receiver": "receiver_count",
    "hourly_network-data_addresses-count_addresses_count_sender": "sender_count",
}
df = df[cols_dict.keys()].rename(cols_dict, axis=1)
df.shape

(11552, 19)

In [12]:
# eda 에서 파악한 차이와 차이의 음수, 양수 여부를 새로운 피쳐로 생성
df = df.assign(
    liquidation_diff=df["long_liquidations"] - df["short_liquidations"],
    liquidation_usd_diff=df["long_liquidations_usd"] - df["short_liquidations_usd"],
    volume_diff=df["buy_volume"] - df["sell_volume"],
    liquidation_diffg=np.sign(df["long_liquidations"] - df["short_liquidations"]),
    liquidation_usd_diffg=np.sign(df["long_liquidations_usd"] - df["short_liquidations_usd"]),
    volume_diffg=np.sign(df["buy_volume"] - df["sell_volume"]),
    buy_sell_volume_ratio=df["buy_volume"] / (df["sell_volume"] + 1),
)
# category, continuous 열을 따로 할당해둠
category_cols: List[str] = ["liquidation_diffg", "liquidation_usd_diffg", "volume_diffg"]
conti_cols: List[str] = [_ for _ in cols_dict.values() if _ not in ["ID", "target", "_type"]] + [
    "buy_sell_volume_ratio",
    "liquidation_diff",
    "liquidation_usd_diff",
    "volume_diff",
]

In [13]:
def shift_feature(
        df: pd.DataFrame,
        conti_cols: List[str],
        intervals: List[int],
) -> List[pd.Series]:
    """
    연속형 변수의 shift feature 생성
    Args:
        df (pd.DataFrame)
        conti_cols (List[str]): continuous colnames
        intervals (List[int]): shifted intervals
    Return:
        List[pd.Series]
    """
    df_shift_dict = [
        df[conti_col].shift(interval).rename(f"{conti_col}_{interval}")
        for conti_col in conti_cols
        for interval in intervals
    ]
    return df_shift_dict


# 최대 24시간의 shift 피쳐를 계산
shift_list = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 24)]
)

In [14]:
# concat 하여 df 에 할당
df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

# 타겟 변수를 제외한 변수를 forwardfill, -999로 결측치 대체
_target = df["target"]
df = df.ffill().fillna(-999).assign(target=_target)

# _type에 따라 train, test 분리
train_df = df.loc[df["_type"] == "train"].drop(columns=["_type"])
test_df = df.loc[df["_type"] == "test"].drop(columns=["_type"])

### Model Training

In [15]:
# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis=1),
    train_df["target"].astype(int),
    test_size=0.2,
    random_state=42,
)

# lgb dataset
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_valid, label=y_valid, reference=train_data)

# lgb params
params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_class": 4,
    "num_leaves": 50,
    "learning_rate": 0.05,
    "n_estimators": 30,
    "random_state": 42,
    "verbose": 0,
}

# lgb train
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
    valid_sets=valid_data,
)

# lgb predict
y_valid_pred = lgb_model.predict(x_valid)
y_valid_pred_class = np.argmax(y_valid_pred, axis=1)

# score check
accuracy = accuracy_score(y_valid, y_valid_pred_class)
auroc = roc_auc_score(y_valid, y_valid_pred, multi_class="ovr")

print(f"acc: {accuracy}, auroc: {auroc}")

/opt/anaconda3/envs/ml-lecture/lib/python3.9/site-packages/lightgbm/engine.py:172: UserWarning:

Found `n_estimators` in params. Will use it instead of argument



acc: 0.4223744292237443, auroc: 0.6145133857950079


In [16]:
# performance 체크후 전체 학습 데이터로 다시 재학습
x_train = train_df.drop(["target", "ID"], axis=1)
y_train = train_df["target"].astype(int)
train_data = lgb.Dataset(x_train, label=y_train)
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
)

/opt/anaconda3/envs/ml-lecture/lib/python3.9/site-packages/lightgbm/engine.py:172: UserWarning:

Found `n_estimators` in params. Will use it instead of argument



### Inference

In [17]:
# lgb predict
y_test_pred = lgb_model.predict(test_df.drop(["target", "ID"], axis=1))
y_test_pred_class = np.argmax(y_test_pred, axis=1)

### Output File Save

In [18]:
# output file 할당후 save 
submission_df = submission_df.assign(target=y_test_pred_class)
submission_df.to_csv("output.csv", index=False)